# Assignment NMT: Intro to NLP

The aim of this homework is to familiarize you with sequence-to-sequence language modeling, specifically using an encoder-decoder model. In this notebook, you are provided with pre-written code for a simple sequence-to-sequence model that already works and learns how to reverse short sequences of numbers.

If you run this whole jupyter notebook, it will learn to reverse short sequences of numbers. Although much of this code you will not be modifying, we recommend reading through it to get a sense of how the model and training works.

This starter code is based on [this tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) by Sean Robertson from the PyTorch website and the COMS course at Columbia University by Professor Kathy McKeown. 

### Overview

Your assignment is to:
 1. adapt this notebook to work on the English-Italian language pair from Tataoeba website
 2. Implement a beam search function
 

Write all your code **in this jupyter notebook**. Cells are provided where you should be implementing your code. 

You do not need to modify any code to train the model. You may modify the `trainIters` function, if you would like to improve how you track progress, or change parameters while training. For example, it can be useful to decrease the teacher-forcing ratio as training progresses.


I would recommend you run this notebook as it is, first. You should be able to run it with the dummy data provided without making ANY modifications (except the cell where the data are loaded). Then, start making your changes as requested.



In [1]:
# You may modify this cell if you like

import random
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import torchtext

In [2]:
# DO NO MODIFY

# this is useful for checking if your code is successfully using the GPU

mydevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mydevice

device(type='cuda')

In [3]:
# DO NOT MODIFY

SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

MAX_LEN = 50

def len_filter(example):
    return len(example.src) <= MAX_LEN and len(example.tgt) <= MAX_LEN

### Load dummy number reversal dataset

In [4]:
#note that my files were stored in google drive and I was using google colab to run this notebook
#you can change this cell to provide local path to load your training and dev data

train_path = 'data/train/data.txt'
dev_path = 'data/dev/data.txt'


In [5]:
# DO NOT MODIFY


src = torchtext.data.Field(
    batch_first=True, 
    include_lengths=True
    )
tgt = torchtext.data.Field(
    batch_first=True, 
    preprocessing = lambda seq: [SOS_TOKEN] + seq + [EOS_TOKEN]
    )

data_train = torchtext.data.TabularDataset(
        path=train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=dev_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

### 1. Load the data (10 points)

Load in the en-it data from http://www.manythings.org/anki/ita-eng.zip

 similar to how the dummy number reversal dataset is loaded above. That is, use the same `torchtext.data.Field` and `torchtext.data.TabularDataset` classes.

In [6]:
# WRITE YOUR CODE FOR LOADING THE ITALIAN<->ENGLISH DATA HERE
#TODO 
#create 6 files, English|Italian data for train|dev|test, one sentence per line.
#you can ignore this cell when you are running the code the first time through on the dummy reversal dataset
import pandas as pd
from sklearn.model_selection import train_test_split

ita_path = "data/anki/ita.txt"

ita_df = pd.read_csv(ita_path,sep='\t', names=["English", "Italian", "Attribution"]).drop(["Attribution"], axis=1)
print("total size:", len(ita_df))

ita_train, ita_test = train_test_split(ita_df, test_size=0.3, shuffle=True, random_state=4111)

ita_train, ita_val = train_test_split(ita_train, test_size=0.3, shuffle=True, random_state=4111)

print("ita_train len:", len(ita_train))
print("ita_val len:", len(ita_val))
print("ita_test len:", len(ita_test))
print("==================")
print("total:", len(ita_train) + len(ita_val) + len(ita_test))

total size: 341554
ita_train len: 167360
ita_val len: 71727
ita_test len: 102467
total: 341554


In [7]:
ita_train_path = "data/train/ita_train.tsv"
ita_val_path = "data/dev/ita_dev.tsv"

ita_train.to_csv(ita_train_path, sep="\t", index=False, header=False)
ita_val.to_csv(ita_val_path, sep="\t", index=False, header=False)

In [8]:
data_train = torchtext.data.TabularDataset(
        path=ita_train_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )
data_dev = torchtext.data.TabularDataset(
        path=ita_val_path, format='tsv',
        fields=[('src', src), ('tgt', tgt)],
        filter_pred=len_filter
    )

Have a look at the vocab and some example data points.

*If you have loaded in the data correctly, the code in the cell below should work without any modification.*

In [9]:
# You may modify this cell if you like

src.build_vocab(data_train, max_size=50000)
tgt.build_vocab(data_train, max_size=50000)
input_vocab = src.vocab
output_vocab = tgt.vocab

print('20 tokens from input vocab:\n', list(input_vocab.stoi.keys())[:20])
print('\n20 tokens from output vocab:\n', list(output_vocab.stoi.keys())[:20])

print('\nnum training examples:', len(data_train.examples))

item = random.choice(data_train.examples)
print('\nexample train data:')
print('src:\n', item.src)
print('tgt:\n', item.tgt)

#showing output below for toy dataset

20 tokens from input vocab:
 ['<unk>', '<pad>', 'I', 'Tom', 'to', 'you', 'a', 'the', 'is', 'in', 'was', "I'm", 'of', 'have', 'You', 'that', 'do', 'be', 'He', 'for']

20 tokens from output vocab:
 ['<unk>', '<pad>', '<eos>', '<sos>', 'Tom', 'di', 'è', 'a', 'non', 'che', 'Io', 'Non', 'un', 'la', 'il', 'ha', 'per', 'in', 'sono', 'una']

num training examples: 167358

example train data:
src:
 ['How', 'many', 'friends', 'do', 'you', 'have?']
tgt:
 ['<sos>', 'Quante', 'amiche', 'avete?', '<eos>']


### Model definition and training functions

In [10]:
# DO NOT MODIFY

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, myinput, hidden):
        embedded = self.embedding(myinput).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=mydevice)

In [11]:
# DO NOT MODIFY

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,
          max_length=MAX_LEN, teacher_forcing_ratio=0.5):
    
    # get an initial hidden state for the encoder
    encoder_hidden = encoder.initHidden()

    # zero the gradients of the optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # get the seq lengths, used for iterating through encoder/decoder
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # create empty tensor to fill with encoder outputs
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    # create a variable for loss
    loss = 0
    
    # pass the inputs through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # create a start-of-sequence tensor for the decoder
    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

    # set the decoder hidden state to the final encoder hidden state
    decoder_hidden = encoder_hidden

    # decide if we will use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input
                
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        
        if use_teacher_forcing:
            decoder_input = target_tensor[di]
        
        if decoder_input.item() == output_vocab.stoi[EOS_TOKEN]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [12]:
# You may modify this cell

def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01, teacher_forcing_ratio=0.5):
    print(f'Running {n_iters} epochs...')
    print_loss_total = 0
    print_loss_epoch = 0

    encoder_optim = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optim = optim.SGD(decoder.parameters(), lr=learning_rate)

    # note batch size of 1, just for simplicity
    # DO NOT INCREASE THE BATCH SIZE
    batch_iterator = torchtext.data.Iterator(
        dataset=data_train, batch_size=1,
        sort=False, sort_within_batch=True,
        sort_key=lambda x: len(x.src),
        device=mydevice, repeat=False)
    

    criterion = nn.NLLLoss()

    for e in range(n_iters):
        batch_generator = batch_iterator.__iter__()
        step = 0
        start = time.time()
        for batch in batch_generator:
            step += 1
            
            # get the input and target from the batch iterator
            input_tensor, input_lengths = getattr(batch, 'src')
            target_tensor = getattr(batch, 'tgt')
            
            # this is because we're not actually using the batches.
            # batch size is 1 and this just selects that first one
            input_tensor = input_tensor[0]
            target_tensor = target_tensor[0]

            loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optim, decoder_optim, criterion, teacher_forcing_ratio=teacher_forcing_ratio)
            print_loss_total += loss
            print_loss_epoch += loss
            

            if step % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                t = (time.time() - start) / 60
                print(f'step: {step}\t avg loss: {print_loss_avg:.2f}\t time for {print_every} steps: {t:.2f} min')
                start = time.time()
        
        print_loss_avg = print_loss_epoch / step
        print_loss_epoch = 0
        print(f'End of epoch {e}, avg loss {print_loss_avg:.2f}')  

###  Create and train a model

In [29]:
# You may modify this cell

hidden_size = 128
encoder1 = EncoderRNN(len(input_vocab), hidden_size).to(mydevice)
decoder1 = DecoderRNN(hidden_size, len(output_vocab)).to(mydevice)

Here are some guidelines for how much training to expect. Note that these *guidelines*; they are not exact.

Only 1 epoch is needed for the number reversal dataset. This produces near-perfect results, and should take less than 5 minutes to run on a CPU.



In [30]:
# You may modify this cell
# but be sure that it prints some indication of how training is progressing

trainIters(encoder1, decoder1, 2, print_every=1000, learning_rate=0.05, teacher_forcing_ratio=0.75)

Running 2 epochs...
step: 1000	 avg loss: 5.80	 time for 1000 steps: 0.25 min
step: 2000	 avg loss: 5.13	 time for 1000 steps: 0.24 min
step: 3000	 avg loss: 4.91	 time for 1000 steps: 0.25 min
step: 4000	 avg loss: 4.79	 time for 1000 steps: 0.25 min
step: 5000	 avg loss: 4.70	 time for 1000 steps: 0.25 min
step: 6000	 avg loss: 4.53	 time for 1000 steps: 0.26 min
step: 7000	 avg loss: 4.51	 time for 1000 steps: 0.25 min
step: 8000	 avg loss: 4.39	 time for 1000 steps: 0.25 min
step: 9000	 avg loss: 4.48	 time for 1000 steps: 0.24 min
step: 10000	 avg loss: 4.30	 time for 1000 steps: 0.22 min
step: 11000	 avg loss: 4.30	 time for 1000 steps: 0.26 min
step: 12000	 avg loss: 4.23	 time for 1000 steps: 0.25 min
step: 13000	 avg loss: 4.23	 time for 1000 steps: 0.25 min
step: 14000	 avg loss: 4.16	 time for 1000 steps: 0.26 min
step: 15000	 avg loss: 4.06	 time for 1000 steps: 0.26 min
step: 16000	 avg loss: 4.14	 time for 1000 steps: 0.26 min
step: 17000	 avg loss: 4.05	 time for 1000 st

In [31]:
# DO NOT MODIFY

def evaluate(encoder, decoder, sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice)

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            next_word = output_vocab.itos[topi.item()]
            decoded_words.append(next_word)
            if next_word == EOS_TOKEN:
                break

            decoder_input = topi.squeeze().detach()

        return decoded_words

Have a look at some generated sequences! This is the fun part.

In [32]:
# You may modify this cell

# This selects 5 random datapoints from the training data and shows the generated sequence

for i in range(5):
    item = random.choice(data_train.examples)
    seq = item.src
    print(seq)
    words = evaluate(encoder1, decoder1, seq)
    print(' '.join(words))
    print()

['Prepare', 'to', 'evacuate.']
<sos> Si a scrivere a dormire. <eos>

["I'll", 'go.']
<sos> La andare. <eos>

['Tom', "wouldn't", 'do', 'it.']
<sos> Tom non lo odia. oggi. <eos>

['Close', 'the', 'drawer.']
<sos> Chiudi il piano. <eos>

['I', 'hear', 'the', 'grass', 'in', 'England', 'is', 'green', 'even', 'in', 'the', 'winter.']
<sos> Io ho sentito dei biblioteca al conto. di più. la terra. di storia. la terra. di storia. la terra. di storia. la terra. di storia. la terra. di storia. la terra. di storia. la terra. di storia. la terra. di più. la terra. di più. la terra. di più.



### Implement beam search (10 points for ITCS 4111 students, 15 points for the ITCS 5111/DSBA 6010 students)

We provide an evaluation function that performs greedy decoding on any input sequence provided. 

(ITCS 4111 students) You must write a new  function that implements beam search for a beam size = 2. 

(ITCS 5111/DSBA 6010 students) You must write a new  function that implements beam search for an arbitrary beam size. Let the beam size be an input parameter to your function. 

In greedy decoding, at each decoding step the most likely word is selected, resulting in one decoded sequence output. 

In beam search, at each decoding step the top k most
likely sequences are selected. Each of these k sequences is then used to generate the next step; the top k next words per sequence are considered (for a total of k ∗ k sequences)
and the top k sequences are selected to take to the next decoding step. At the end, you have k decoded sequence outputs.


In [33]:
# credit for original source of code is https://github.com/budzianowski/PyTorch-Beam-Search-Decoding/blob/master/decode_beam.py
# I modified their algorithm and optimised it a bit

In [34]:
class BeamSearchNode:
    def __init__(self, parent_node, hidden_state, word_id, score):
        self.parent = parent_node
        self.h = hidden_state
        self.word_id = word_id
        self.score = score
        
        if self.parent == None:
            self.length = 1
        else:
            self.length = len(self.parent) +1
        
    def __len__(self):
        return self.length
    
    def eval(self, alpha=1.0):
        reward = 0
        ## Add here a function for shaping a reward

        return self.score / float(self.length - 1 + 1e-6) + alpha * reward


In [35]:
import operator
from queue import PriorityQueue

def beam_decode(encoder, decoder, sentence, max_length=MAX_LEN, width=2):
    input_tensor = torch.tensor([input_vocab.stoi[word] for word in sentence], device=mydevice)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=mydevice)

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] += encoder_output[0, 0]

    decoder_input = torch.tensor([[output_vocab.stoi[SOS_TOKEN]]], device=mydevice) #input is Start of Sentence token

    decoder_hidden = encoder_hidden
    
    endnodes = []

    for idx in range(input_tensor.size(0)):
        #print(idx)
        #print(decoder_input)
        
        node = BeamSearchNode(None, decoder_hidden, decoder_input, 0)
        nodes = PriorityQueue()
        
        nodes.put((-node.eval(), node))
                
        while True:
            if nodes.qsize() > 2000: break
            
            #get current best node
            score, current_node = nodes.get()
            decoder_input = current_node.word_id
            decoder_hidden = current_node.h
            
            if current_node.word_id.item() == EOS_TOKEN and current_node.parent != None:
                endnodes.append((score, current_node))
                break
                
            decoder_output, decoder_hidden = decoder1(decoder_input, decoder_hidden)
            
            log_probability, indexes = torch.topk(decoder_output, width)
            
            for k in range(width):
                decoded_t = indexes[0][k].view(1, -1)
                log_p = log_probability[0][k].item()
                
                node = BeamSearchNode(current_node, decoder_hidden, decoded_t, current_node.score + log_p)
                score = -node.eval()
                nodes.put((score, node))                
        
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(1)]

        decoded_ids = []
        sentence = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):

            decoded_ids.append(n.word_id)
            # back trace
            while n.parent != None:
                n = n.parent
                decoded_ids.append(n.word_id)

            decoded_ids = decoded_ids[::-1]
            
    for decoded_id in decoded_ids:
        word = output_vocab.itos[decoded_id.item()]
        if word != SOS_TOKEN and word != EOS_TOKEN:
            sentence.append(word)            
    return sentence
            

In [52]:
item = random.choice(data_train.examples)
seq = item.src
print("English:",' '.join(seq))
words = beam_decode(encoder1, decoder1, seq, width=3)
print("Beam Search Translation:",' '.join(words))
print()

English: Let Tom answer.
Beam Search Translation: Lasciate Tom

